Face Detection with Deep Learning

Locating a face in a photograph : finding the coordinate of the face in the image.
Localization - demarcating the extent of the face, often via a bounding box around the face.

Method 01 : Cascade Classifiers + AdaBoost

In [ ]:
# plot photo with detected faces using opencv cascade classifier
import cv2
import os
import uuid
import time

def detect_face(file_name):
	# load the photograph
	pixels = cv2.imread(file_name)
	# load the pre-trained model
	classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
	# perform face detection
	bboxes = classifier.detectMultiScale(pixels,1.05,3)

	# print bounding box for each detected face
	for box in bboxes:
		# extract
		x, y, width, height = box
		x2, y2 = x + width, y + height
		# draw a rectangle over the pixels
		cv2.rectangle(pixels, (x, y), (x2, y2), (0,0,255), 1)
	# show the image
	# cv2.imshow('face detection', pixels)
	#Detect the number of faces detected
	print("Number of Faces Detected in {} : ".format(file_name),len(bboxes))

capture = cv2.VideoCapture(0)
img_num=1
while img_num<=25:
	print('Collecting image {}'.format(img_num))
	ret,frame =capture.read()
	img_name = os.path.join('./data/images_haar/',f'{str(img_num)}.jpg')
	if ret == 0:
		print("Frame Not Captured")
	else:
		cv2.imwrite(img_name, frame)
		img_num+=1
		# cv2.imshow('Frame: '+str(img_num), frame)
		time.sleep(0.5)
	# keep the window open until we press a key
	detect_face(img_name)
	
	if cv2.waitKey(1) & 0xFF==ord('q'): break

capture.release()
# close the window 
cv2.destroyAllWindows()

In the above method,  
- Certain boxes would be recognized as faces, although they are not.
- No detection of inverted faces, tilted faces

Method 02 : Using Multi-Task Cascaded Convolutional Neural Network (MTCNN)

In [ ]:
# extract and plot each detected face in a photograph
from matplotlib import pyplot
from matplotlib.patches import Rectangle
from matplotlib.patches import Circle
from mtcnn.mtcnn import MTCNN

# draw each face separately
def draw_faces(filename, result_list):
	# load the image
	data = pyplot.imread(filename)
	# plot each face as a subplot
	for i in range(len(result_list)):
		# get coordinates
		x1, y1, width, height = result_list[i]['box']
		x2, y2 = x1 + width, y1 + height
		# define subplot
		pyplot.subplot(1, len(result_list), i+1)
		pyplot.axis('off')
		# plot face
		pyplot.imshow(data[y1:y2, x1:x2])
	# show the plot
	pyplot.show()
def detect_face_using_mtcnn(filename):
	#---------------------Check For Exception Handling 
	# load image from file
	pixels = pyplot.imread(filename)
	# create the detector, using default weights
	detector = MTCNN()
	# detect faces in the image
	faces = detector.detect_faces(pixels)
	# display faces on the original image
	print("Number of faces: ",len(faces))
	# draw_faces(filename, faces)

import cv2
import os
import uuid
import time

capture = cv2.VideoCapture(0)
img_num=1
while img_num<=10:
	init = time.time()
	print('Collecting image {}'.format(img_num))
	captured,frame =capture.read()
	img_name = os.path.join('./data/images_mtcnn/',f'{str(img_num)}.jpg')
	if not captured:
		print("Frame Not Captured")
	else:
		cv2.imwrite(img_name, frame)
		img_num+=1
		# cv2.imshow('Frame: '+str(img_num), frame)
		time.sleep(0.5)
	# keep the window open until we press a key
		detect_face_using_mtcnn(img_name)
	
	if cv2.waitKey(1) & 0xFF==ord('q'): break
	fps = 1/(time.time() - init)
	print('Fps: {}'.format(fps))

capture.release()
# close the window 
cv2.destroyAllWindows()


In the above MTCNN technique,
- Detection of inverted faces and tilted faces (A few tilted images could be omitted)